In [1]:
MODEL_VERSION = "001"

In [2]:
import torch
torch.set_float32_matmul_precision('medium')
from torchvision import transforms, models
from torchvision.transforms import Resize, ToTensor, Normalize

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from code.classifier import PneumoniaClassifier
from code.dataloader import PneumoniaDataset
from code.custom_checkpoint import CustomModelCheckpoint
from code.project_globals import TEST_DIR, TRAIN_DIR, VAL_DIR
import pathlib


In [3]:
#transform = transforms.Compose([transforms.Resize((60,60)),transforms.ToTensor()])
# Define transformations for the dataset
transform = transforms.Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
#print(TRAIN_DIR.as_posix())
# Create dataset instances with proper arguments
train = PneumoniaDataset(root_dir=TRAIN_DIR.as_posix(), transform=transform)
test = PneumoniaDataset(root_dir=TEST_DIR.as_posix(), transform=transform)
val = PneumoniaDataset(root_dir=VAL_DIR.as_posix(), transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val, batch_size=32, shuffle=False)

# TensorBoard logger setup
logger = TensorBoardLogger("tb_logs", name="pneumonia_classifier")

checkpoint_callback = CustomModelCheckpoint(
    monitor='val_loss',         # Metric to monitor
    dirpath='../checkpoints',    # Directory to save the checkpoints
    filename='best_model-{epoch:02d}-{val_loss:.2f}',  # Save format
    save_top_k=3,               # Only keep the 3 best models
    mode='min'                  # Minimize val_loss
)

# Model setup
model = PneumoniaClassifier(
    backbone=models.resnet50(weights='ResNet50_Weights.DEFAULT'),
    transfer_learning=True
)

# Trainer with TensorBoard logger
trainer = pl.Trainer(
    max_epochs=10,
    accelerator="gpu",
    devices=1,
    logger=logger,  # Attach TensorBoard logger
    log_every_n_steps=1,  # Log metrics after every step
    callbacks=[checkpoint_callback]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Nov 15 18:02:18 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.90                 Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 Ti   WDDM  |   00000000:02:00.0  On |                  N/A |
|  0%   41C    P8             11W /  285W |    3662MiB /  12282MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# Train the model
trainer.fit(model, train_loader, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type           | Params | Mode 
-------------------------------------------------------------
0 | accuracy          | BinaryAccuracy | 0      | train
1 | feature_extractor | Sequential     | 23.5 M | train
2 | classifier        | Linear         | 4.1 K  | train
-------------------------------------------------------------
4.1 K     Trainable params
23.5 M    Non-trainable params
23.5 M    Total params
94.049    Total estimated model params size (MB)
152       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 163/163 [02:30<00:00,  1.08it/s, v_num=20]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 163/163 [02:31<00:00,  1.08it/s, v_num=20, val_acc_epoch=0.914]Learning Rate after epoch 0: 0.001


C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028


Epoch 1: 100%|██████████| 163/163 [02:35<00:00,  1.05it/s, v_num=20, val_acc_epoch=0.914, train_acc_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 163/163 [02:36<00:00,  1.04it/s, v_num=20, val_acc_epoch=0.957, train_acc_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 163/163 [02:35<00:00,  1.05it/s, v_num=20, val_acc_epoch=0.963, train_acc_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 163/163 [02:34<00:00,  1.06it/s, v_num=20, val_acc_epoch=0.968, train_acc_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 163/163 [02:36<00:00,  1.04it/s, v_num=20, val_acc_epoch=0.972, train_acc_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 6: 100%|██████████| 163/163 [02:31<00:00,  1.07it/s, v_num=20, val_acc_epoch=0.974, train_acc_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 7: 100%|██████████| 163/163 [02:35<00:00

`Trainer.fit` stopped: `max_epochs=10` reached.


Learning Rate after epoch 9: 0.00025
Epoch 9: 100%|██████████| 163/163 [02:36<00:00,  1.04it/s, v_num=20, val_acc_epoch=0.977, train_acc_epoch=0.000]


In [7]:
checkpoint_path = '../checkpoints/best_model-epoch=07-val_loss=0.12.ckpt'
checkpoint = torch.load(checkpoint_path)

# Access the metadata
metadata = checkpoint.get("metadata", {})
print(f"Checkpoint metadata: {metadata}")

# Restore the model state
model.load_state_dict(checkpoint["state_dict"])

C:\Users\aszab\AppData\Local\Temp\ipykernel_2044\973869640.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


FileNotFoundError: [Errno 2] No such file or directory: '../checkpoints/best_model-epoch_10-val_loss_0.25.ckpt'